## Packages

In [1]:
import numpy as np
import sys
import os
import pandas as pd
import boto3

# Data storage
from sqlalchemy import create_engine  # SQL helper
import psycopg2 as psql  # PostgreSQL DBs

sys.path.append("..")

In [2]:
import keys
import data_fcns as dfc

## Options

In [3]:
# So that items in pandas columns don't show truncated values 
pd.set_option('display.max_colwidth', -1)

## Create Official item factors matrix or dataframe

In [4]:
item_factors_df = pd.read_pickle('support_data/item_factors_20190922.pkl')

In [5]:
item_factors_df.head()

,id,features
0,60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]"
1,80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]"
2,110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]"
3,200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]"
4,240,"[0.0, 0.2235216647386551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07523377239704132, 0.15706036984920502, 0.02980225533246994, 0.0, 0.0, 0.0, 0.03824526444077492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17709434032440186, 0.0, 0.0, 0.098581463098526, 0.0, 0.1592487245798111, 0.0, 0.1055234745144844, 0.059581972658634186, 0.0, 0.1497793197631836, 0.07032154500484467, 0.0, 0.10109743475914001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11361673474311829, 0.0, 0.0, 0.0, 0.09224706143140793, 0.2049531638622284, 0.0, 0.0, 0.0, 0.11741948872804642, 0.0, 0.0004655775846913457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.042575541883707047, 0.0,

## Get comics data from DB

In [6]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret',
                               'aws_ps_flatiron.json')
secret_path_aws

aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [7]:
# Instantiate cursor
cur = conn.cursor()

In [8]:
#  Count records.
query = """
    SELECT * FROM comics 
"""

# Execute the query
cur.execute(query)

conn.commit()

In [9]:
# Check results
comics_pdf = pd.DataFrame(cur.fetchall())
comics_pdf.columns = [col.name for col in cur.description]

In [10]:
comics_pdf.head()

,comic_id,comic_title,img_url,lrg_img_url
0,235,Amber Blake (IDW),https://comrx.s3-us-west-2.amazonaws.com/covers/amber_blake.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/amber_blake.jpg
1,358,Armor Hunters Bloodshot (Other),https://comrx.s3-us-west-2.amazonaws.com/covers/armor_hunters_bloodshot.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/armor_hunters_bloodshot.jpg
2,500,A Year of Marvels Unbeatable (Marvel),https://comrx.s3-us-west-2.amazonaws.com/covers/a_year_of_marvels_unbeatable.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/a_year_of_marvels_unbeatable.jpg
3,711,Beef (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/beef.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/beef.jpg
4,726,Belladonna (Avatar),https://comrx.s3-us-west-2.amazonaws.com/covers/belladonna.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/belladonna.jpg


In [11]:
comics_pdf.shape

(7202, 4)

In [12]:
comics_pdf.head()

,comic_id,comic_title,img_url,lrg_img_url
0,235,Amber Blake (IDW),https://comrx.s3-us-west-2.amazonaws.com/covers/amber_blake.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/amber_blake.jpg
1,358,Armor Hunters Bloodshot (Other),https://comrx.s3-us-west-2.amazonaws.com/covers/armor_hunters_bloodshot.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/armor_hunters_bloodshot.jpg
2,500,A Year of Marvels Unbeatable (Marvel),https://comrx.s3-us-west-2.amazonaws.com/covers/a_year_of_marvels_unbeatable.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/a_year_of_marvels_unbeatable.jpg
3,711,Beef (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/beef.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/beef.jpg
4,726,Belladonna (Avatar),https://comrx.s3-us-west-2.amazonaws.com/covers/belladonna.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/belladonna.jpg


### Need to reduce to comics with counts >= 20

In [13]:
comic_ids = pd.read_json('raw_data/comic_ids.json',orient='records', lines=True )

In [14]:
comic_ids.shape

(790, 1)

In [15]:
comics_pdf_small = comics_pdf.merge(comic_ids, left_on='comic_id', right_on='comic_id', how='inner', )

In [16]:
comics_pdf.shape

(7202, 4)

In [17]:
comics_pdf_small.shape

(790, 4)

In [18]:
comics_pdf_small.head()

,comic_id,comic_title,img_url,lrg_img_url
0,1316,Conan the Barbarian (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg
1,1634,Dceased (DC),https://comrx.s3-us-west-2.amazonaws.com/covers/dceased.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/dceased.jpg
2,2400,From Under Mountains (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/from_under_mountains.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/from_under_mountains.jpg
3,2986,Head Lopper (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/head_lopper.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/head_lopper.jpg
4,6348,Tmnt Ongoing (IDW),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg


## Merge item factors and comics

In [19]:
# Change column names for item_factors_df
item_factors_df.columns = ['comic_id', 'features']

In [20]:
combo = item_factors_df.merge(comics_pdf_small, left_on='comic_id', right_on='comic_id', how='inner', )

In [21]:
combo.set_index(['comic_id'], inplace=True)

In [22]:
combo.head()

,features,comic_title,img_url,lrg_img_url
comic_id,,,,
60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]",8house (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/8house.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/8house.jpg
80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]",Action Comics Annual (DC),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg
110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]",Adventure Time (Boom),https://comrx.s3-us-west-2.amazonaws.com/covers/adventure_time.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/adventure_time.jpg
200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]",All Star Batman (DC),https://comrx.s3-us-west-2.amazonaws.com/covers/all_star_batman.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/all_star_bat

In [23]:
comics_pdf.loc[comics_pdf['comic_id']==20]

,comic_id,comic_title,img_url,lrg_img_url
105,20,1 For $1 Conan the Barbarian (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg,https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg


In [24]:
combo.shape

(790, 4)

In [25]:
combo.drop(['img_url'], axis=1, inplace=True)

In [26]:
combo.shape

(790, 3)

In [27]:
combo.head()

,features,comic_title,lrg_img_url
comic_id,,,
60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]",8house (Image),https://comrx.s3-us-west-2.amazonaws.com/covers_large/8house.jpg
80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]",Action Comics Annual (DC),https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg
110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]",Adventure Time (Boom),https://comrx.s3-us-west-2.amazonaws.com/covers_large/adventure_time.jpg
200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]",All Star Batman (DC),https://comrx.s3-us-west-2.amazonaws.com/covers_large/all_star_batman.jpg
240,"[0.0, 0.2235216647386551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07523377239704132, 0.15706036984920502, 0.02980225533246994, 0.0, 0.0, 0.0, 0.03824526444077492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17709434032440186, 0.0, 0.0, 0.098581463098526, 0.0, 0.15924872457

In [28]:
combo.columns = ['features', 'comic_title', 'img_url']

In [29]:
combo.head()

,features,comic_title,img_url
comic_id,,,
60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]",8house (Image),https://comrx.s3-us-west-2.amazonaws.com/covers_large/8house.jpg
80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]",Action Comics Annual (DC),https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg
110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]",Adventure Time (Boom),https://comrx.s3-us-west-2.amazonaws.com/covers_large/adventure_time.jpg
200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]",All Star Batman (DC),https://comrx.s3-us-west-2.amazonaws.com/covers_large/all_star_batman.jpg
240,"[0.0, 0.2235216647386551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07523377239704132, 0.15706036984920502, 0.02980225533246994, 0.0, 0.0, 0.0, 0.03824526444077492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17709434032440186, 0.0, 0.0, 0.098581463098526, 0.0, 0.159248724579811

## Create pickle file

In [43]:
#combo.to_pickle('support_data/comics_factors_201908.pkl')

In [30]:
combo.to_pickle('support_data/comics_factors_20190922.pkl')

# Test 2019.08.12 fix

In [63]:
# Read new pickle

In [32]:
cf_new = pd.read_pickle('support_data/comics_factors_20190922.pkl')

In [33]:
# Read old pickle 

In [34]:
cf_old = pd.read_pickle('support_data/comics_factors.pkl')

In [35]:
cf_new.shape

(790, 3)

In [37]:
cf_old.shape

(6028, 3)

In [38]:
cf_new.head()

,features,comic_title,img_url
comic_id,,,
60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]",8house (Image),https://comrx.s3-us-west-2.amazonaws.com/covers_large/8house.jpg
80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]",Action Comics Annual (DC),https://comrx.s3-us-west-2.amazonaws.com/covers_large/_no_cover_.jpg
110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]",Adventure Time (Boom),https://comrx.s3-us-west-2.amazonaws.com/covers_large/adventure_time.jpg
200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]",All Star Batman (DC),https://comrx.s3-us-west-2.amazonaws.com/covers_large/all_star_batman.jpg
240,"[0.0, 0.2235216647386551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07523377239704132, 0.15706036984920502, 0.02980225533246994, 0.0, 0.0, 0.0, 0.03824526444077492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17709434032440186, 0.0, 0.0, 0.098581463098526, 0.0, 0.159248724579811

In [39]:
cf_old.head()

,features,comic_title,img_url
comic_id,,,
10,"[-0.7526867389678955, -0.21263617277145386, -1.1569229364395142, 0.19937847554683685, 0.16002951562404633]",13th Artifact One Sho (Topcow),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
20,"[-0.3515812158584595, 0.4757572114467621, -1.2307846546173096, 0.5941579341888428, -0.11247903108596802]",1 For $1 Conan the Barbarian (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
30,"[0.1806577742099762, -0.48153993487358093, -0.9925048351287842, -0.04510089382529259, -0.6085895299911499]",21st Century Tank Girl (Other),https://comrx.s3-us-west-2.amazonaws.com/covers/21st_century_tank_girl.jpg
40,"[-0.06164746731519699, -0.23286470770835876, -0.20415398478507996, 0.3465690016746521, -1.2049529552459717]",4001 Ad (Other),https://comrx.s3-us-west-2.amazonaws.com/covers/4001_ad.jpg
50,"[-0.4028661251068115, -0.3713889420032501, -1.1736090183258057, 0.15386144816875458, 0.5422862768173218]",68 Homefront (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/68_homefront.jpg
